In [1]:
import numpy as np
import pandas as pd
import zipfile as zfile
from functools import reduce

In [2]:
### thanks to MarcH https://stackoverflow.com/a/14981125 for this function to print to stderr: 
from __future__ import print_function
import sys

def errprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)
###

In [3]:
def openChapters(filename):
    """
    This function open a zip or a csv file containing raw trade data from customs.go.jp.
    The following columns are expected in the file:
    - "Year", containing a 4 digit year number
    - "Country, a 3-digit code for each country
    - "HS", a code used to identify the category of the imported/exported product.
    Also, we expect 36 columns created as a couple "Type-Mon" where "type" is one among
    "Quantity1", "Quantity2" or "Value", while "Mon" represents the first three letters
    (capitalized) of a specific month, e.g. "Jan", "Aug" etc. 
    In case the file is a zip with multiple csv files, the function opens and merges
    the files into a single DataFrame object, which the function returns.
    If the file is a single csv, return the DataFrame built from it.
    """
    def merge(frame1,frame2):
        return frame1.append(frame2,ignore_index=True)
    
    pieces = []
    trade_types = {"Year":int,"HS":object, "Country":object}
    if filename[-4:] == ".zip":
        with zfile.ZipFile(filename) as z:
            for f in z.namelist():
                if f[-4:] == ".csv":
                    with z.open(f) as piece:
                        piece = pd.read_csv(piece, dtype=trade_types)
                        if "Quantity1-Apl" in piece.columns.tolist():
                            piece.rename(columns={"Quantity1-Apl":"Quantity1-Apr",\
                                                  "Quantity2-Apl":"Quantity2-Apr",\
                                                 "Value-Apl":"Value-Apr"},\
                                         inplace=True)
                        pieces.append(piece)
            return reduce(merge, pieces)
        
    elif filename[-4:] == ".csv":
        single_file = pd.read_csv(filename, dtype=trade_types)
        return single_file

In [124]:
def meltMonths(df):
    '''
    This function melts the 36 columns "unit-month" into three:
    - "date", a Y-M-D date format with the last day of the month
    - "type", with one of the three possible values "Quantity1", "Quantity2", "Value"
    - "measure", with the value associated to the type
    '''
    # this dictionary will be useful to convert the months from words to digits
    month_dict = {"Jan":"-01", "Feb":"-02","Mar":"-03","Apr":"-04",\
             "May":"-05", "Jun":"-06","Jul":"-07","Aug":"-08",\
             "Sep":"-09", "Oct":"-10","Nov":"-11","Dec":"-12"}
    
    # a very lousy check but a check nontheless...
    if len(df.columns) < 20:
        errprint("meltMonths: the dataframe provided does not have monthly columns!")
        return df
        
    columns = df.columns.tolist()
    # not interested in yearly values - can recover them later anyway
    melting = df.drop(columns=[c for c in columns if "-Year" in c])
    melting = melting.drop(columns=["Exp or Imp"])
        
    ids = ["Year", "HS", "Country", "Unit1", "Unit2"]
    # melting all values but the above
    to_melt = melting.columns.tolist()
    for c in ids:
        to_melt.remove(c)
        
    melted = pd.melt(melting, id_vars=ids, value_vars=to_melt, value_name = "measure")
    
    # splitting the month and the unit in two different columns
    # note: the comprehension list is actually faster than str.split and str.extract
    melted[["type","month"]] = pd.DataFrame([r.split('-') for r in list(melted["variable"])])
    melted = melted.drop(columns=["variable"])
    
    # we can start reducing the size of the DF by erasing the rows that have Quantity 1
    # or Quantity 2 without a measure.
    no_qty1 = melted[(melted["type"] == "Quantity1") & (melted["Unit1"] == "  ")].index
    no_qty2 = melted[(melted["type"] == "Quantity2") & (melted["Unit2"] == "  ")].index
    melted = melted.drop(no_qty1).drop(no_qty2)
    
    # now, merging creating a datetime column from the year and the month
    melted["month"] = melted["month"].replace(month_dict)
    melted["date"]=melted.Year.map(str)+melted.month
    
    # we can now get rid of year, day and month:
    melted = melted.drop(columns=["Year","month"])
    
    return melted
        

In [5]:
def meltUnits(df):
    '''
    This function executes an essential transformation for trade files, as
    in the original file up to three units (unit1, unit2, and the implicit
    "yen" value) are present for each row.
    We want to have a table that is  normalized  with respect to units,
    that is, a row will have a column "unit" that will contain the exact 
    unit the measure is in (e.g. Kg, Ton, No. of items, 1000 yen...) and a
    column for the numeric value ("measure").
    '''
    # a very lousy check but a check nontheless...
    if "Unit2" not in df.columns.tolist():
        errprint("meltUnits: the dataframe provided does not have units!")
        return df
    
    melted = df.copy()
    # this or dropna won't work
    melted["unit"] = np.nan
    
    # creating the masks that will be useful to reconstruct a single unit column
    q1_mask = (melted["type"] == "Quantity1")
    q2_mask = (melted["type"] == "Quantity2")
    val_mask = (melted["type"] == "Value")
    
    # not entirely satisfied with using np.where for multiple values on the same column
    melted["unit"] = np.where(q1_mask, melted["Unit1"], melted["unit"])
    melted["unit"] = np.where(q2_mask, melted["Unit2"], melted["unit"])
    melted["unit"] = np.where(val_mask, "000s JPY", melted["unit"])
    
    # these columns have been replaced by "unit", won't be useful anymore
    melted.drop(columns=["Unit1","Unit2","type"], inplace=True)
    
    # drop the rows with an empty unit (happens when unit2 is empty)
    # empty units are actually a string of two spaces: "  ".
    melted = melted.dropna(subset=["unit"])
    empty_id = melted[melted["unit"]=="  "].index
    melted = melted.drop(empty_id)
    
    return melted

In [6]:
def normalizeData(file, to_db_file = None, to_dataframe = None):
    '''
    Open a trade file (csv, zip) and normalizes it. If a pd.Dataframe reference
    or a csv db file (priority on DF) is provided, adds the normalized file to it
    Returns a pd.DataFrame.
    '''
    unnormalized = openChapters(file)
    month_melted = meltMonths(unnormalized)
    unit_melted = meltUnits(month_melted)

    if to_dataframe != None:
        # we are giving priority to existing dataframes.
        # if both a DF and a file are specified, only the DF will be considered.
        if to_db_file != None:
            errprint("warning: both DataFrame and csv file were provided to normalizeData, but only the DataFrame will be used.")
        base = to_dataframe
        # merge to the existing dataframe
        pass
    elif to_db_file != None:
        base = openChapters(to_db_file)
        #merge to the existing file
        pass
    else:
        # no file or DataFrame to merge with
        return unit_melted

In [7]:
path = "trade2015-2019.zip"
df = openChapters(path)
df.iloc[150:200]

,Exp or Imp,Year,HS,Country,Unit1,Unit2,Quantity1-Year,Quantity2-Year,Value-Year,Quantity1-Jan,...,Value-Sep,Quantity1-Oct,Quantity2-Oct,Value-Oct,Quantity1-Nov,Quantity2-Nov,Value-Nov,Quantity1-Dec,Quantity2-Dec,Value-Dec
150,2,2015,'010620031',205,NO,KG,130,8,646,0,...,0,0,0,0,130,8,646,0,0,0
151,2,2015,'010620031',207,NO,KG,228,43,6475,0,...,0,0,0,0,20,4,1013,167,27,1175
152,2,2015,'010620031',213,NO,KG,1820,119,17861,57,...,4667,81,23,1481,2,19,2209,585,28,4869
153,2,2015,'010620031',225,NO,KG,25,5,2641,0,...,0,0,0,0,0,0,0,0,0,0
154,2,2015,'010620031',227,NO,KG,265,10,1424,0,...,0,0,0,0,0,0,0,0,0,0
155,2,2015,'010620031',238,NO,KG,1973,108,16589,0,...,1899,285,16,4859,0,0,0,20,2,295
156,2,2015,'010620031',245,NO,KG,1268,104,7510,0,...,381,0,0,0,0,0,0,0,0,0
157,2,2015,'010620031',302,NO,KG,1450,60,5223,0,...,0,0,0,0,241,21,773,0,0,0
158,2,2015,'010620031',304,NO,KG,19754,990,70680,2313,...,4675,1676,69,5755,1117,25,3203,2286,198,7280
159,2,2015,'010620031',305,NO,KG,78,56,2402,0,...,1324,0,0,0,0,0,0,38,5,739


In [112]:
df1 = meltMonths(df)
df1[df1["date"].isna()].count()

HS         0
Country    0
Unit1      0
Unit2      0
measure    0
type       0
date       0
dtype: int64

In [113]:
df2 = meltUnits(df1)
df2.iloc[150:200]

,HS,Country,measure,date,unit
200,'010631000',225,1,2015-1,NO
201,'010631000',227,0,2015-1,NO
202,'010631000',245,0,2015-1,NO
203,'010631000',302,0,2015-1,NO
204,'010631000',304,0,2015-1,NO
205,'010631000',407,11,2015-1,NO
206,'010632000',112,79,2015-1,NO
207,'010632000',117,163,2015-1,NO
208,'010632000',204,0,2015-1,NO
209,'010632000',205,0,2015-1,NO


In [115]:
df2_nozero = df2[df2["measure"]!=0]
df2_nozero = df2_nozero.sort_values(by=["Country","HS","unit","date"]).reset_index(drop=True)
df2_nozero.to_csv("trade2015-2019incompl_nozero.zip", index=False, compression="infer")